<a href="https://colab.research.google.com/github/kamalzada/Machine-Learning-Projects/blob/master/GI_GL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#Importing useful libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests
import seaborn as sns 
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [12]:
URL = f'http://www.mendosa.com/gilists.htm'

In [93]:
def request_and_parse(url):
  request = requests.get(url) 
  soup = bs(request.text, 'lxml')
  foods_gi = soup.find_all('td')
request_and_parse(URL)     

In [106]:
products_gi = foods_gi[:137]

In [111]:
prod_gi = []
prods = []
gis = []
for value in products_gi[6:]:
  if 'align' not in str(value):
      prod_gi.append(value) 
for key, value in enumerate(prod_gi):
  if key%2==0:
    prods.append(value.text)
  else:
    gis.append(value.text)  

In [117]:
#Convert into a dataframe
df = pd.DataFrame(data = prods, columns = ['products'])
df['glucose_index(GI)'] = pd.Series(gis).to_frame() 
df['glucose_index(GI)'] = df['glucose_index(GI)'].str[:-2] 

In [118]:
#View
df.tail()

,products,glucose_index(GI)
57,Rice crackers/crisps,87
58,Fructose,15
59,Sucrose,65
60,Glucose,103
61,Honey,61


In [ ]:
from google.colab import files
df.to_csv('Products_GI.csv')
files.download('Products_GI.csv') 

In [ ]:
#Now, let's scrape the other table consisting of over 2000 foods and convert it into a dataset

In [13]:
reqs = requests.get(URL) 
soup = bs(reqs.text, 'lxml')
food_gi = soup.find_all('tr') 

In [14]:
food_gi = food_gi[82:4574] 

In [15]:
product_rows = []

for product in food_gi:
  if '±' in str(product.text) and 'mean' not in str(product.text):
    product_rows.append(product.text)  

In [22]:
df1 = pd.Series(product_rows).to_frame().rename(columns={0: 'all_columns'})  
df1.head() 

,all_columns
0,"1 Banana cake, made with sugar 47±8 60 14"
1,"2 Banana cake, made without sugar 55±10 60 12"
2,4 Chocolate cake made from packet mix 38±3 11...
3,"5 Cupcake, strawberry-iced (Squiggles, 73±12 ..."
4,6 Lamingtons (sponge dipped in chocolate and 8...


In [34]:
df1['prod_gin'] = df1['all_columns'].str.split('±', expand=True)[0]
df1['glo'] = df1['all_columns'].str.split('±', expand=True)[1]

In [45]:
#Importing useful functions
%run Utility_tools.ipynb
values_stripper(df1) 

Categorical features' values are stripped


In [92]:
df1['prod_gin'] = df1['prod_gin'].str[-3:]
values_stripper(df1)  
df1 = df1.replace('8 7', '7').replace('2 3', '3').replace('d 7', '7').replace('® 3', '3').replace('ean', np.nan).replace('2 7', '7').replace('2 8', '8').replace('1 2', '2').replace(') 6', '6')
df1['glycemic_index(GI)'] = df1['prod_gin'] 

Categorical features' values are stripped


In [51]:
df1['glycemic_load'] = df1['glo'].str[-2:]

In [56]:
df1['glycemic_load'].unique()

array(['14', '12', '20', '19', '25', '9', '13', '24', '6', '11', '15',
       '16', '17', '22', '38', '23', '32', '18', '8', '4', '5', '1', '7',
       '29', '10', '3', '21', '2', '27', '26', '-', '30', '36', '28',
       '44', '37', '31', '46', '40', '33', '35', '34', '39', '0', '42',
       '43', 'd.', '51', '55', '49', '58', '41', '52', '47'], dtype=object)

In [63]:
df1 = df1.replace('d.', np.nan).replace('-', np.nan)

In [80]:
df1['product_name'] = df1['all_columns'].str[:-11]
df1['product_name'] = df1['product_name'].str.split(' ', 1, expand=True)[1]

In [94]:
df1.drop(['all_columns', 'prod_gin', 'glo'], axis=1, inplace=True)

In [96]:
df1.isna().sum()

glycemic_load         16
product_name           0
glycemic_index(GI)     1
dtype: int64

In [99]:
duplicate_check_remove(df1)

Number of duplicate rows before: 22
Number of duplicate rows now: 0


In [103]:
df1 = df1[['product_name', 'glycemic_index(GI)', 'glycemic_load']]

In [107]:
#Final view
df1.head()

,product_name,glycemic_index(GI),glycemic_load
0,"Banana cake, made with sugar",47,14
1,"Banana cake, made without sugar",55,12
2,Chocolate cake made from packet mix,38,20
3,"Cupcake, strawberry-iced (Squiggles,",73,19
4,Lamingtons (sponge dipped in chocolate and,87,25


In [108]:
from google.colab import files
df1.to_csv('Food_GI_GL.csv')
files.download('Food_GI_GL.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>